In [2]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main

You should consider upgrading via the 'D:\Youssef\Lab\FineTuneWhisper-KR\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'D:\Youssef\Lab\FineTuneWhisper-KR\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
  Using cached https://download.pytorch.org/whl/cu116/torch-1.12.0%2Bcu116-cp310-cp310-win_amd64.whl (2388.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.2.0.dev0 requires torch>=1.13.0, but you have torch 1.12.0+cu116 which is incompatible.


In [18]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.
Token is valid.
Your token has been saved in your configured git credenti

In [3]:
from datasets import load_dataset, DatasetDict

Zeroth_Korean = DatasetDict()

Zeroth_Korean["train"] = load_dataset("Bingsu/zeroth-korean", split="train", use_auth_token=True, cache_dir="D:/Youssef/.cache/huggingface/datasets")
Zeroth_Korean["test"] = load_dataset("Bingsu/zeroth-korean", split="test", use_auth_token=True, cache_dir="D:/Youssef/.cache/huggingface/datasets")

print(Zeroth_Korean)

Using custom data configuration Bingsu--zeroth-korean-787ca68963c66467
Found cached dataset parquet (D:/Youssef/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--zeroth-korean-787ca68963c66467/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Using custom data configuration Bingsu--zeroth-korean-787ca68963c66467
Found cached dataset parquet (D:/Youssef/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--zeroth-korean-787ca68963c66467/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 22263
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 457
    })
})


In [4]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2", language="Korean", task="transcribe")

In [5]:
print(Zeroth_Korean["train"][0])

{'audio': {'path': None, 'array': array([-3.05175781e-05,  0.00000000e+00, -3.05175781e-05, ...,
        0.00000000e+00,  0.00000000e+00, -6.10351562e-05]), 'sampling_rate': 16000}, 'text': '인사를 결정하는 과정에서 당 지도부가 우 원내대표 및 원내지도부와 충분한 상의를 거치지 않은 채 일방적으로 인사를 했다는 불만도 원내지도부를 중심으로 흘러나왔다'}


In [6]:
def dataset_prep(batch):
    # load audio
    audio = batch["audio"]

    # calculate log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids

    return batch

In [7]:
# applying data preparation func to all training examples using .map
Zeroth_Korean = Zeroth_Korean.map(dataset_prep, remove_columns=Zeroth_Korean.column_names["train"], num_proc=1)

  0%|          | 0/22263 [00:00<?, ?ex/s]

  0%|          | 0/457 [00:00<?, ?ex/s]

In [8]:
print(Zeroth_Korean)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 22263
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 457
    })
})


In [9]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

         # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [10]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [11]:
import evaluate

metric = evaluate.load("wer", cache_dir="D:/Youssef/.cache/huggingface/evaluate")

In [12]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [14]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2", load_in_8bit=True, device_map="auto", cache_dir="D:/Youssef/.cache/huggingface/hub")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


D:\Youssef\Lab\FineTuneWhisper-KR\venv\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\Youssef\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: Loading binary D:\Youssef\Lab\FineTuneWhisper-KR\venv\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: Loading binary D:\Youssef\Lab\FineTuneWhisper-KR\venv\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
a

D:\Youssef\Lab\FineTuneWhisper-KR\venv\lib\site-packages\bitsandbytes\cuda_setup\main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('/matplotlib_inline.backend_inline')}
  warn(msg)
D:\Youssef\Lab\FineTuneWhisper-KR\venv\lib\site-packages\bitsandbytes\cuda_setup\main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
  warn(msg)
D:\Youssef\Lab\FineTuneWhisper-KR\venv\lib\site-packages\bitsandbytes\cuda_setup\main.py:136: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
  warn(msg)
D:\Youssef\Lab\FineTuneWhisper-KR\venv\lib\site-packages\bitsandbytes\cuda_setup\main.py:136: UserWarning: WARNING: No GPU detected! Check your CUDA paths. Proceeding to load CPU-only library...
  warn(msg)


RuntimeError: 
        CUDA Setup failed despite GPU being available. Inspect the CUDA SETUP outputs above to fix your environment!
        If you cannot find any issues and suspect a bug, please open an issue with detals about your environment:
        https://github.com/TimDettmers/bitsandbytes/issues

In [12]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from peft import prepare_model_for_training

model = prepare_model_for_training(model, output_imbedding_layer="proj_out")

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

In [13]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-Large-v2-KR",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=25,
    remove_unused_columns=False,
    label_names=["labels"],
)

In [14]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=Zeroth_Korean["train"],
    eval_dataset=Zeroth_Korean["test"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
)
model.config.use_cache = False  # silence the warnings. Should re-enable for inference!!!

D:\Youssef\Lab\FineTuneWhisper-KR\./whisper-small-KR is already a clone of https://huggingface.co/byoussef/whisper-small-KR. Make sure you pull the latest changes with `repo.git_pull()`.
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [15]:
processor.save_pretrained(training_args.output_dir)

Feature extractor saved in ./whisper-small-KR\preprocessor_config.json
tokenizer config file saved in ./whisper-small-KR\tokenizer_config.json
Special tokens file saved in ./whisper-small-KR\special_tokens_map.json
added tokens file saved in ./whisper-small-KR\added_tokens.json


In [ ]:
trainer.train()

In [16]:
kwargs = {
    "dataset_tags": "Bingsu/zeroth-korean",
    "dataset": "Zeroth-Korean",
    "language": "ko",
    "model_name": "Whisper Large V2 PEFT KR - BYoussef",
    "finetuned_from": "openai/whisper-large-v2",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [17]:
trainer.push_to_hub(**kwargs)

Saving model checkpoint to ./whisper-small-KR
Configuration saved in ./whisper-small-KR\config.json
Configuration saved in ./whisper-small-KR\generation_config.json
Model weights saved in ./whisper-small-KR\pytorch_model.bin
Feature extractor saved in ./whisper-small-KR\preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 32.0k/922M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.61k/3.61k [00:00<?, ?B/s]

Upload file runs/Feb22_14-36-02_DESKTOP-EVRCGBF/events.out.tfevents.1677044316.DESKTOP-EVRCGBF: 100%|#########…

Upload file runs/Feb23_11-52-57_DESKTOP-EVRCGBF/1677120788.2072656/events.out.tfevents.1677120788.DESKTOP-EVRC…

Upload file runs/Feb23_11-50-11_DESKTOP-EVRCGBF/events.out.tfevents.1677120618.DESKTOP-EVRCGBF: 100%|#########…

Upload file runs/Feb23_11-50-11_DESKTOP-EVRCGBF/1677120618.3952792/events.out.tfevents.1677120618.DESKTOP-EVRC…

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/byoussef/whisper-small-KR
   1474173..3286e22  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Automatic Speech Recognition', 'type': 'automatic-speech-recognition'}, 'dataset': {'name': 'Zeroth-Korean', 'type': 'Bingsu/zeroth-korean', 'args': 'config: kr, split: test'}}
To https://huggingface.co/byoussef/whisper-small-KR
   3286e22..0299e6c  main -> main



'https://huggingface.co/byoussef/whisper-small-KR/commit/3286e22af19f792a746d2d14389bdc24874dbfcd'